# Battleship game

In [1]:
import numpy as np
import pdb
import copy

In [2]:
col_alpha = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T"]

In [3]:
class Coordinate():
    def __init__(self,row=0,col=0):
        self.row = int(row)
        self.col = int(col)
    def __repr__(self):
        return "({}, {})".format(self.row+1, col_alpha[self.col])

In [4]:
class Ship():
    def __init__(self,size=2, orientation='-', head=None, ship_id = 0):
        self.size = size
        self.head = head # head of the ship is placed on this coordinate
        self.life = size # number of hits that need to be sustained to sink this ship
        self.orientation = orientation # '=' or '||' or '\\' stands for horz, vert or diag
        self.ship = self.orientation*self.size
        self.id = ship_id
    def is_sink(self):
        return self.life<=0
    def is_float(self):
        return self.life>0
    def __repr__(self):
        return str(self.id) + ": " + str(self.ship) + " with head at {}".format(self.head)

In [19]:
class Board():
    def __init__(self,board_size=9):
        self.board_size = board_size
        # self.board = [['_']*self.board_size]*self.board_size 
        self.board = []
        for i in range(self.board_size):
            row = []
            for j in range(self.board_size):
                row.append('_')
            self.board.append(row)
        self.masked_board = copy.deepcopy(self.board) # 'o' for sea_shot, 'X' for ship_attacked
        self.ship_id_board = copy.deepcopy(self.board) # id 1 for 1st ship and henceforth
        self.ships = 0
        self.ship_list = []
    def print_board(self, masked=False):
        string = ''
        string += "   " + " ".join(col_alpha[:self.board_size])+'\n'
        cnt = 1
        if not masked:
            # print("no mask")
            board = self.board
        else:
            # print("mask")
            board = self.masked_board
        # pdb.set_trace()
        for row in board:
            if len(str(cnt)) == 1:
                string += " " + str(cnt) + " " + " ".join(row) + '\n'
            else:
                string += str(cnt) + " " + " ".join(row) + '\n'
            cnt += 1
        string +=  "Ships left: " + str(self.ships)
        print(string)
    def __repr__(self):
        string = ''
        string += "   " + " ".join(col_alpha[:self.board_size])+'\n'
        cnt = 1
        board = self.board
        for row in board:
            if len(str(cnt)) == 1:
                string += " " + str(cnt) + " " + " ".join(row) + '\n'
            else:
                string += str(cnt) + " " + " ".join(row) + '\n'
            cnt += 1
        string +=  "Ships left: " + str(self.ships)
        return string
    def occupied(self, s):
        size = s.size
        orientation = s.orientation
        curr = s.head
        while((size!=0) & (curr!=None)):
            if self.board[curr.row][curr.col]!='_':
                return True
            size-=1
            curr = self.next_coordinate(curr,orientation)
        return False
    
    def next_coordinate(self, curr, orientation):
        if orientation == '||':
            new_row, new_col = curr.row+1, curr.col
        elif orientation == '\\':
            new_row, new_col =  curr.row+1, curr.col+1
        else: # orientation == '='
            new_row, new_col = curr.row, curr.col+1
        if new_row<0 | new_row>=self.board_size | new_col<0 | new_col>=self.board_size:
            return False
        else:
            return Coordinate(new_row, new_col)
        
    def add_ship(self, size, orientation='=', head=Coordinate(0,0)):
        id = len(self.ship_list)
        s = Ship(size, orientation, head, id)
        if self.occupied(s):
            print("Sorry can't add this ship here")
            return False
        else:
            size = s.size
            # orientation = s.orientation
            curr = s.head
            while(size!=0):
                # pdb.set_trace()
                # print(curr)
                self.board[curr.row][curr.col] = s.orientation
                self.ship_id_board[curr.row][curr.col] = s.id
                # print(self.__repr__())
                curr = self.next_coordinate(curr, orientation)
                size-=1
            self.ships+=1
            self.ship_list.append(s)
            return True
    def respond(self, shot='1A'):
        row = int(shot[0])-1
        col = col_alpha.index(shot[1])
        if row<0 | row>=self.board_size | col<0 | col>=self.board_size:
            print("Invalid shot")
        else:
            if (self.board[row][col]=='_'):
                self.masked_board[row][col] = 'o'
                self.board[row][col] = 'o'
            elif (self.board[row][col]=='X'):
                print("Already attacked here")
                pass
            else:
                ship_id = self.ship_id_board[row][col]
                self.masked_board[row][col] = 'X'
                self.board[row][col] = 'X'
                # Identify the ship and check if it has sunk
                ship = self.ship_list[ship_id]
                ship.life-=1
                if ship.is_sink():
                    self.ships-=1
        

In [26]:
class Game:
    def __init__(self, num_ships, against_computer=False):
        self.players=2
        self.boards = [Board(), Board()]
        self.num_ships = num_ships
        self.in_progress=True
        self.setup()
        self.play()
    def setup(self):
        print("**********************")
        print("***   Battleship   ***")
        print("**********************")
        print("")
        print("Each player is going to have {} ship(s) each".format(self.num_ships))
        for i in range(self.players):
            print("")
            print("Player {} - Add your ships now".format(i+1))
            # for j in range(self.num_ships):
            j=0
            while(j<self.num_ships):
                print("Ship {}".format(j+1))
                size = int(input("Ship size: "))
                orientation = input("Ship orientation (Choose from: = OR || OR \\) : ")
                head_shot = input("Ship head location (example: 1A) \\\\\: ")
                row = int(head_shot[0])-1
                col = col_alpha.index(head_shot[1])
                head = Coordinate(row,col)
                j += self.boards[i].add_ship(size, orientation, head)
        print("")
    def play(self):
        turn = 0
        print("**********************")
        print("Let the game begin !!")
        while(self.in_progress):
            player = turn % 2
            other_player = 1 - player
            print("**********************")
            print("Turn {}".format(turn+1))
            print("Player {}'s masked board: ".format(other_player+1))
            self.boards[other_player].print_board(masked=True)
            response = input("Player {} attack(example: 1A) : ".format(player+1))
            self.boards[other_player].respond(response)
            print("Player {}'s masked board post attack: ".format(other_player+1))
            self.boards[other_player].print_board(masked=True)
            print("Player {}'s board: ".format(player+1))
            self.boards[player].print_board(masked=False)
            # Check if game won
            if self.boards[other_player].ships==0:
                self.in_progress=False
                print("Player {} won!".format(player+1))
            # Check if wanna quit
            q = input("Wanna quit (q or Q) : ")
            if (q=='q') | (q=='Q'):
                self.in_progress=False
            turn+=1
        print("Quitting")

In [27]:
Game(1)

**********************
***   Battleship   ***
**********************

Each player is going to have 1 ship(s) each

Player 1 - Add your ships now
Ship 1
Ship size: 4
Ship orientation (Choose from: = OR || OR \) : \
Ship head location (example: 1A) \\\: 2B

Player 2 - Add your ships now
Ship 1
Ship size: 3
Ship orientation (Choose from: = OR || OR \) : ||
Ship head location (example: 1A) \\\: 5D

**********************
Let the game begin !!
**********************
Turn 1
Player 2's masked board: 
   A B C D E F G H I
 1 _ _ _ _ _ _ _ _ _
 2 _ _ _ _ _ _ _ _ _
 3 _ _ _ _ _ _ _ _ _
 4 _ _ _ _ _ _ _ _ _
 5 _ _ _ _ _ _ _ _ _
 6 _ _ _ _ _ _ _ _ _
 7 _ _ _ _ _ _ _ _ _
 8 _ _ _ _ _ _ _ _ _
 9 _ _ _ _ _ _ _ _ _
Ships left: 1
Player 1 attack(example: 1A) : 4A
Player 2's masked board post attack: 
   A B C D E F G H I
 1 _ _ _ _ _ _ _ _ _
 2 _ _ _ _ _ _ _ _ _
 3 _ _ _ _ _ _ _ _ _
 4 o _ _ _ _ _ _ _ _
 5 _ _ _ _ _ _ _ _ _
 6 _ _ _ _ _ _ _ _ _
 7 _ _ _ _ _ _ _ _ _
 8 _ _ _ _ _ _ _ _ _
 9 _ _ _ _ _ _ _

Player 2 attack(example: 1A) : 2B
Player 1's masked board post attack: 
   A B C D E F G H I
 1 _ _ _ _ _ _ _ _ _
 2 o X _ _ _ _ _ _ _
 3 _ _ X _ _ _ _ _ _
 4 _ _ _ X _ _ _ _ _
 5 _ _ _ _ X _ _ _ _
 6 _ _ _ _ _ _ _ _ _
 7 _ _ _ _ _ _ _ _ _
 8 _ _ _ _ _ _ _ _ _
 9 _ _ _ _ _ _ _ _ _
Ships left: 0
Player 2's board: 
   A B C D E F G H I
 1 _ _ _ _ _ _ _ _ o
 2 o _ _ _ _ _ _ _ _
 3 _ _ _ _ _ _ _ _ _
 4 o _ _ _ _ _ _ _ _
 5 _ _ _ || _ _ _ _ _
 6 _ _ _ || _ _ _ _ _
 7 _ _ _ X _ _ _ _ _
 8 _ _ _ _ _ _ _ _ _
 9 o _ _ _ _ _ _ _ _
Ships left: 1
Player 2 won!
Wanna quit (q or Q) : n
Quitting
